# Optimise Iterative Backtesting - Part 6

In [ ]:
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np
from itertools import product
from IPython.display import display, clear_output

In [ ]:
class Backtester():
    def __init__(self, ticker, ema_short_period, ema_long_period, amount):
        self.symbol = ticker
        self.ema_s = ema_short_period
        self.ema_l = ema_long_period
        self.result = None
        self.trades = 0
        self.units = 0
        self.logs = pd.DataFrame()
        self.data = pd.DataFrame()
        self.position = 0
        self.initial_balance = amount
        self.current_balance = amount

    def reset_variables(self):
        self.result = None
        self.trades = 0
        self.units = 0
        self.logs = pd.DataFrame()
        self.data = pd.DataFrame()
        self.position = 0
        self.current_balance = self.initial_balance
        
    def set_params(self, ema_short_period, ema_long_period):
        self.ema_s = ema_short_period
        self.ema_l = ema_long_period
        
    def fetch_data(self):
        # API, Database, csv file
        data = pd.read_csv('eurusd.csv')
        data['date'] = pd.to_datetime(data['date'])
        data.set_index('date', drop=True, inplace=True)
        data['return'] = np.log(data.close/data.close.shift(1))
        data['ema_short'] = data['close'].ewm(span=self.ema_s, adjust=False).mean()
        data['ema_long'] = data['close'].ewm(span=self.ema_l, adjust=False).mean()
        self.data = data
    def plot_price(self):
        self.data[['close', 'ema_short', 'ema_long']].plot(figsize=(14, 7))

    def access_data(self, bar):
        date_value = self.data.index[bar].date()
        price = float(self.data['close'].iloc[bar])
        ema_short = float(self.data['ema_short'].iloc[bar])
        ema_long = float(self.data['ema_long'].iloc[bar])
        return date_value, price, ema_short, ema_long
        
    def plot_return(self):
        self.data['return'].plot(figsize=(14, 7))

    def get_balance(self):
        return self.current_balance
    def get_current_position(self, bar):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        current_position = self.units * price
        print(f" Current Position Value: {current_position}")

    def get_current_NAV(self, bar):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        current_position = self.units * price
        current_NAV = self.current_balance + current_position
        print(f" Current NAV: {current_NAV}")
        
    def buy_instrument(self, bar, qty):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        self.units = self.units + qty
        self.current_balance = self.current_balance - (price * qty)
        self.trades = self.trades + 1
        #print(f"{date_value} | BUY {qty} @ $: {round(price, 2) }")
    def sell_instrument(self, bar, qty):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        self.units = self.units - qty
        self.current_balance = self.current_balance + (price * qty)
        self.trades = self.trades + 1        
        #print(f"{date_value} | SELL {qty} @ $: {round(price, 2) }")
        
    def close_positions(self, bar):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        if self.position != 0:
            #print("\n")
            #print('$$$ - Closing positions Finally - $$$')
            if self.position == 1:
                self.sell_instrument(bar = bar, qty = abs(self.units))
            elif self.position == -1:
                self.buy_instrument(bar = bar, qty = abs(self.units))
            self.position = 0
        performance = ((self.current_balance - self.initial_balance) / self.initial_balance) * 100
        # print("\n")
        # print("------------------- Summary -----------------")
        # print(f"Performance: {round(performance, 2)} %")
        # self.net_balance = self.get_balance()
        # print(f"Current Balance: {self.net_balance}")
        # print(f"Number of Trades: {self.trades}")
        return round(performance, 2)
        
    def test_strategy_one(self, ema_short_period, ema_long_period):
        self.set_params(ema_short_period, ema_long_period)
        self.reset_variables()
        self.fetch_data()
        qty = 50000
        for bar in range(len(self.data) -1):
            close = self.data['close'].iloc[bar]
            ema_s = self.data['ema_short'].iloc[bar]
            ema_l = self.data['ema_long'].iloc[bar]
            if ema_s > ema_l:
                if self.position == 0:
                    self.buy_instrument(bar = bar, qty = qty)
                elif self.position == -1:
                    self.buy_instrument(bar = bar, qty = qty * 2)

                if self.position != 1:
                    balance = self.get_balance()
                    #print(f"Current Balance: {balance}")
                    self.position = 1
                    #print(f"Current Units: {self.units}")
            elif ema_l > ema_s:
                if self.position == 0:
                    self.sell_instrument(bar = bar, qty = qty)
                elif self.position == 1:
                    self.sell_instrument(bar = bar, qty =  qty * 2)
                    
                if self.position != -1:
                    balance = self.get_balance()
                    #print(f"Current Balance: {balance}")
                    self.position = -1
                    #print(f"Current Units: {self.units}")
        return self.close_positions(-1)

In [ ]:
obj = Backtester(ticker = 'NIFTY', ema_short_period = 38, ema_long_period = 160, amount = 70000)

In [ ]:
obj.test_strategy_one(38, 160)

In [ ]:
range_short_sma = np.arange(38, 42, 1)
range_long_sma = np.arange(155, 160 ,1)
combinations = list(product(range_short_sma, range_long_sma))
results = []
i = 0
for comb in combinations:
    i+= 1
    #print(*comb)
    results.append(obj.test_strategy_one(*comb))
    clear_output(wait=True)
    print(i, ' of ', len(combinations))

In [ ]:
np.max(results)

In [ ]:
np.min(results)

In [ ]:
dfFinal = pd.DataFrame(data=combinations, columns=["range_sma", "range_lma"])
dfFinal["Performance (points)"] = results

In [ ]:
dfFinal.nlargest(10, "Performance (points)")

In [ ]:
dfFinal.nsmallest(10, "Performance (points)")